In [1]:
import pickle
import numpy as np
from numpy.linalg import norm

feature_list=np.array(pickle.load(open("new_embeddings.pkl","rb"))) # list converted into array
filenames=pickle.load(open("new_filenames.pkl","rb"))

print(np.array(feature_list).shape)

(27616, 2048)


In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import GlobalMaxPooling2D
from tensorflow.keras.applications.resnet50 import ResNet50,preprocess_input

In [3]:
model= ResNet50(weights="imagenet",include_top=False,input_shape=(224,224,3)) # we get weights that has trained on imagenet dataset
model.trainable=False    # we are not going to train the model because we using the imagenet datset which is already trained

In [4]:
model = Sequential([
    model,     #pass the resnet50 model in sequential layer
    GlobalMaxPooling2D()   # we added our layer globalmaxpooling2d instead of top layer . It gives 2048 features
])

In [18]:
img=image.load_img("sample/2977.jpg",target_size=(224,224))  # i convert the image size in 224x224
img_array=image.img_to_array(img)
expand_img_array=np.expand_dims(img_array,axis=0)   # into betch
preprocessed_img=preprocess_input(expand_img_array)
result=model.predict(preprocessed_img).flatten()     # model give the features of the given image 
normalized_result=result/norm(result)   
        

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step


In [19]:
# to calculate nearest features 
from sklearn.neighbors import NearestNeighbors

In [20]:
neighbours=NearestNeighbors(n_neighbors=7,algorithm='brute',metric="euclidean")
neighbours.fit(feature_list)   
neighbours.kneighbors([normalized_result])

(array([[0.        , 0.525823  , 0.62829387, 0.63947737, 0.64005429,
         0.64180726, 0.64866257]]),
 array([[   73,    80, 21991,  9838,  8072,    43,  8094]], dtype=int64))

it give distance and indices

In [21]:
distance,indices=neighbours.kneighbors([normalized_result])
print(indices)

[[   73    80 21991  9838  8072    43  8094]]


In [22]:
for file in indices[0]:
    print(filenames[file])

10000images\2977.jpg
10000images\2978.jpg
10000images\57424.jpg
10000images\42394.jpg
10000images\3987.jpg
10000images\2974.jpg
10000images\3989.jpg


In [23]:
import cv2

In [24]:
for file in indices[0]:
    pre_img=cv2.imread(filenames[file])
    cv2.imshow("output",cv2.resize(pre_img,(300,300),pre_img))
    cv2.waitKey(0)